In [1]:
import sys
import tskit
import pandas as pd
import time
import os
sys.path.append('../../scripts/')
from spaceNNtime_templates import *
from tensorflow.python.client import device_lib

In [2]:
sim, exp, nam, met, snp, pre, lay, dro, typ, cov, std, err, los, nfe, nla, wti, wsp, wsa, nod  = ["europe", "9", "loss", "5", "1.0", "sNNt", "10", "0.25", "gt", "0", "0", "0.0", "mse", "None", "Norm1", "1", "10000", "None", "64"]
snp = float(snp)
lay = int(lay)
dro = float(dro)
cov = float(cov)
std = float(std)
err = float(err)
nod = int(nod)
wti = float(wti)
wsp = float(wsp)

In [3]:
ts            = tskit.load("../../data/{sim}/tree.trees".format(sim = sim))
metadata      = pd.read_csv("../../data/{sim}/metadata/{met}.txt".format(sim = sim, met = met), delimiter = "\t")

In [4]:
if nam not in ["loss", "reference", "downsample", "sampling", "snp_density", "prediction", "n_nodes", "dropout", "layers"]:
    cov_file_path = "/home/moicoll/spaceNNtime/sandbox/{sim}/{exp}/coverage.txt".format(sim = sim, exp = exp)
    if os.path.isfile(cov_file_path):
        cov = pd.read_table(cov_file_path)[["co1", "co2"]].to_numpy().reshape(-1)
    else:
        ploidy = 2
        cov = simGL.depth_per_haplotype(rng = np.random.default_rng(1234), mean_depth = cov/ploidy, std_depth = std/ploidy, n_hap = metadata.shape[0]*ploidy, ploidy = ploidy)
        pd.DataFrame({"ind" : metadata["ind_id"],
                    "co1" : cov.reshape(-1, 2)[:, 0],
                    "co2" : cov.reshape(-1, 2)[:, 1]}).to_csv(cov_file_path, mode='w', header=True, sep = "\t", index = False)

In [5]:
if wsa == "None":
    wsa = np.ones(metadata.shape[0])
elif wsa == "coverage":
    wsa = cov.reshape(-1, 2).sum(axis = 1)

input         = get_input(ts, metadata, snp, typ, cov, err)
output        = get_output(pre, metadata)
print("input shape:", input.shape)
print(input)
print("output shape:", output.shape)
print(output)

print("Getting travaltes")
tra_val_tes   = get_tra_val_tes(metadata["ind_id"].to_numpy(), file = "../../sandbox/{sim}/{met}/tra_val_tes_{met}.json".format(sim = sim, met = met))
print("Done")

input shape: (1344, 1500)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
output shape: (1500, 3)
[[5.36054544e+01 3.05757414e+01 3.86700000e+03]
 [3.73355222e+01 5.82851504e+01 4.07600000e+04]
 [3.78928083e+01 6.28855568e+01 3.53750000e+04]
 ...
 [5.11460970e+01 9.06849563e+00 9.90100000e+03]
 [5.47534622e+01 3.29253557e+01 8.86100000e+03]
 [3.80293684e+01 6.11433546e+01 3.97470000e+04]]
Getting travaltes
Done


In [64]:
?tf.keras.layers.Normalization

Init signature: tf.keras.layers.Normalization(*args, **kwargs)
Docstring:     
Feature-wise normalization of the data.

This layer will coerce its inputs into a distribution centered around
0 with standard deviation 1. It accomplishes this by precomputing the mean and
variance of the data, and calling `(input - mean) / sqrt(var)` at runtime.

What happens in `adapt()`: Compute mean and variance of the data and store
them as the layer's weights. `adapt()` should be called before `fit()`,
`evaluate()`, or `predict()`.

Args:
    axis: Integer, tuple of integers, or None. The axis or axes that should
      have a separate mean and variance for each index in the shape. For
      example, if shape is `(None, 5)` and `axis=1`, the layer will track 5
      separate mean and variance values for the last axis. If `axis` is set to
      `None`, the layer will normalize all elements in the input by a scalar
      mean and variance. Defaults to -1, where the last axis of the input is
      assumed

In [66]:
def normalizer(nor, input_shape):
    if nor == "None":
        return tf.keras.layers.Normalization(input_shape=[input_shape, ], axis = None, mean = 0, variance = 1), 0, 1
    elif nor == "Norm0":
        
        return tf.keras.layers.Normalization(input_shape=[input_shape, ], axis = None)
    elif nor == "Norm1":
        return tf.keras.layers.Normalization(input_shape=[input_shape, ], axis = -1)

In [69]:
nla = "Norm0"

In [70]:

i = 0
tf.random.set_seed(1234)
i = str(i)
print("Processing batch {}".format(i), flush = True)
norm_features   = normalizer(nor = nfe, input_shape = input.T.shape[1])
if nfe != "None":
    norm_features.adapt(input[:, tra_val_tes[i]["tra"]].T)
norm_labels     = normalizer(nor = nla, input_shape = output.shape[1])
if nla != "None":
    norm_labels.adapt(output[tra_val_tes[i]["tra"], :])

Processing batch 0


In [71]:
nla

'Norm0'

In [72]:
(output[tra_val_tes[i]["tra"], :])

array([[3.73355222e+01, 5.82851504e+01, 4.07600000e+04],
       [3.78928083e+01, 6.28855568e+01, 3.53750000e+04],
       [5.62063243e+01, 3.56497041e+01, 4.79900000e+03],
       ...,
       [5.11460970e+01, 9.06849563e+00, 9.90100000e+03],
       [5.47534622e+01, 3.29253557e+01, 8.86100000e+03],
       [3.80293684e+01, 6.11433546e+01, 3.97470000e+04]])

In [78]:
norm_labels(output[tra_val_tes[i]["tra"], :]).numpy().mean(axis = None)

-1.6047972e-08

In [38]:
norm_labels(output[tra_val_tes[i]["tra"], :]).numpy().std(axis = 0)

array([1.        , 0.9999995 , 0.99999964], dtype=float32)

In [43]:
spaceNNtime(output_shape  = output.shape[1], 
                        norm          = norm_features, 
                        dropout_prop  = dro, 
                        l             = lay, 
                        n             = nod, 
                        loss_function = los, 
                        w_time        = wti, 
                        w_space       = wsp)

In [ ]:
[[ 1.6063088   0.17639016 -1.1250343 ]
 [ 0.7640381  -1.5982187  -0.93265826]
 [-1.2564942   0.46277282  1.337214  ]]
(3, 3)
m = [   42.62135567    40.77918183 25356.05048255]
[8.76563130e+00 1.64173166e+01 1.44648645e+04]
[[5.67016665e+01 4.36750349e+01 9.08258126e+03]
 [4.93186318e+01 1.45407197e+01 1.18652752e+04]
 [3.16073911e+01 4.83766697e+01 4.46986698e+04]]

In [88]:
a = np.array([[ 1.6063088,0.17639016,-1.1250343 ],
 [ 0.7640381 , -1.5982187 , -0.93265826],
 [-1.2564942,0.46277282,1.337214  ]])
m = np.array([[   42.62135567,40.77918183,25356.05048255]])
s = np.array([[8.76563130e+00,1.64173166e+01,1.44648645e+04]])

In [89]:
(a*s)+m

array([[5.67016664e+01, 4.36750349e+01, 9.08258178e+03],
       [4.93186320e+01, 1.45407194e+01, 1.18652751e+04],
       [3.16073908e+01, 4.83766697e+01, 4.46986698e+04]])

In [90]:
m.shape

(1, 3)